<a href="https://colab.research.google.com/github/BriacLEGUILLOU/Impact_score/blob/master/Impact_score_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
pd.set_option('display.max_rows', 60)

In [ ]:
def obtention_exp10(nb):
  """ Permet de convertir un nombre de la façon suivante :
  1e6 -> 6
  - 1e6 -> -6 """
  signe = np.sign(nb)
  nb = np.absolute(nb)
  if nb == 0:
    return 0
  else :
    return math.exp(nb * math.log(10)) * signe
  
obtention_exp10(6)

1000000.0000000013

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Chargement des données
# L'utilisateur peut sélectionner la catégories APE souhaitée
# Il est nécessaire d'avoir accès au dossier 'Export données par catégorie' de Briac
ape_cat0 = '03'
data_log10 = pd.read_excel('/content/gdrive/My Drive/Impact Score/Export données par catégorie/data{}.xlsx'.format(ape_cat0), index_col=0) # donnée logarithmic
data = pd.concat([data_log10.iloc[:,0],data_log10.iloc[:,1:69].applymap(obtention_exp10), data_log10.iloc[:,69:-1]], axis=1) # donnée réelle

# Réalisation de graphiques

In [ ]:
data

,year,"Autres impôts, taxes et versements assimilés",Ventes de marchandises,Production vendue biens,Production vendue services,Chiffres d’affaires nets,Production stockée,Production immobilisée,Subventions d’exploitation,"Reprises sur amortissements et provisions, transfert de charges",Autres produits,Total des produits d’exploitation,Achats de marchandises (y compris droits de douane),Variation de stock (marchandises),Achats de matières premières et autres approvisionnements,Variation de stock (matières premières et approvisionnements),Autres achats et charges externes,"Impôts, taxes et versements assimilés",Salaires et traitements,Charges sociales,Autres charges,Total des charges d’exploitation,Résultat d'exploitation,Bénéfice attribué ou perte transférée,Perte supportée ou bénéfice transféré,Produits financiers de participations,Produits des autres valeurs mobilières et créances de l’actif immobilisé,Autres intérêts et produits assimilés,Reprises sur provisions et transferts de charges financier,Différences positives de change,Produits nets sur cessions de valeurs mobilières de placement,Total des produits financiers,Dotations financières sur amortissements et provisions,Intérêts et charges assimilées,Différences négatives de change,Charges nettes sur cessions de valeurs mobilières de placement,Total des charges financières,Résultat financier,Résultat en cours avant impôts,Produits exceptionnels sur opérations de gestion,...,Total des produits exceptionnels,Charges exceptionnelles sur opérations de gestion,Charges exceptionnelles sur opérations en capital,Dotations exceptionnelles aux amortissements et provisions,Total des charges exceptionnelles,Résultat exceptionnel,Participation des salariés aux résultats de l’entreprise,Impôts sur les bénéfices,Total des produits,Total des charges,Bénéfices ou perte (Total des produits ‐ Total des charges),Impôts différés (compte de résultat),Résultat net des sociétés mises en équivalence,Résultat net des entreprises intégrées,Résultat Groupe (Résultat net consolidé),Part des intérêts minoritaires (Résultat hors groupe),Résultat net part du groupe (part de la société mère),Rémunération d’intermédiaires et honoraires (hors rétrocessions),"Location, charges locatives et de copropriété",Effectif moyen du personnel,Sous‐traitance,Personnel extérieur à l’entreprise,"Rétrocessions d’honoraires, commissions et courtages",Taxe professionnelle,Montant de la TVA. collectée,Total TVA. déductible sur biens et services,Dividendes,siren,denomination,enthic,postal_code,town,optimisation_fiscale,ape,ape_name,len,ape_cat0,ape_cat1,ape_cat2,ape_cat3
6570,2013,NaN,NaN,NaN,29160.0,29160.0,NaN,NaN,NaN,421.0,NaN,29581.0,NaN,NaN,NaN,NaN,2450.0,2221.0,NaN,NaN,NaN,9871.0,19710.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,1709.0,-1704.0,18006.0,NaN,...,NaN,NaN,NaN,NaN,2260120.0,-2260120.0,NaN,NaN,29586.0,2271700.0,-2242114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321435471,AQUACULTURE DE LA DROME,391,26700,PIERRELATTE,False,0322Z,Aquaculture en eau douce,5,3,2,2,Z
6571,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4860.0,NaN,4860.0,NaN,NaN,NaN,NaN,5095.0,NaN,NaN,NaN,4860.0,10295.0,-5435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1884.0,NaN,NaN,1884.0,-1884.0,-7319.0,NaN,...,NaN,52784.0,NaN,NaN,52784.0,-52784.0,NaN,NaN,4860.0,64963.0,-60103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321435471,AQUACULTURE DE LA DROME,391,26700,PIERRELATTE,False,0322Z,Aquaculture en eau douce,5,3,2,2,Z
6572,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,137.0,NaN,NaN,NaN,NaN,4354.0,NaN,NaN,NaN,NaN,4439.0,-4301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1813.0,NaN,NaN,1813.0,-1813.0,-6114.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,6251.0,-6114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321435471,AQUACULTURE DE LA DROME,391,26700,PIERRELATTE,False,0322Z,Aquaculture en eau douce,5,3,2,2,Z
6573,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4810